In [1]:
import os
import functools
from itertools import groupby
import operator
from collections import defaultdict

import numpy as np
import pandas as pd

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

from codon_hamiltonian import *

---
# Amino_acid_to Codon

Input: Amino acid sequence

In [3]:
"""
def fragmenting_amino_acid_seq(Amino_acid_seq, length_frag, ith):
    return Amino_acid_seq[length_frag * (ith-1):length_frag * (ith)]
"""

In [2]:
#spike_sars2_seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT'
A0A2U1LIM9 = 'MQSTTSVKLSPFDLMTALLNGKVSFDTSNTSDTNIPLAVFMENRELLMILTTSVAVLIGCVVVLVWRRSSSAAKKAAESPVIVVPKKVTEDEVDDGRKKVTVFFGTQTGTAEGFAKALVEEAKARYEKAVFKVIDLDDYAAEDDEYEEKLKKESLAFFFLATYGDGEPTDNAARFYKWFTEGEEKGEWLEKLQYAVFGLGNRQYEHFNKIAKVVDEKLVEQGAKRLVPVGMGDDDQCIEDDFTAWKELVWPELDQLLRDEDDTSVATPYTAAVAEYRVVFHDKPETYDQDQLTNGHAVHDAQHPCRSNVAVKKELHSPLSDRSCTHLEFDISNTGLSYETGDHVGVYVENLSEVVDEAEKLIGLPPHTYFSVHTDNEDGTPLGGASLPPPFPPCTLRKALASYADVLSSPKKSALLALAAHATDSTEADRLKFLASPAGKDEYAQWIVASHRSLLEVMEAFPSAKPPLGVFFASVAPRLQPRYYSISSSPKFAPNRIHVTCALVYEQTPSGRVHKGVCSTWMKNAVPMTESQDCSWAPIYVRTSNFRLPSDPKVPVIMIGPGTGLAPFRGFLQERLAQKEAGTELGTAILFFGCRNRKVDFIYEDELNNFVETGALSELVTAFSREGATKEYVQHKMTQKASDIWNLLSEGAYLYVCGDAKGMAKDVHRTLHTIVQEQGSLDSSKAELYVKNLQMAGRYLRDVW'

test_amino_seq = fragmenting_amino_acid_seq(A0A2U1LIM9, 3, 1)
test_amino_seq

'MQS'

Enumerating all poissible codons of an input

In [3]:
codon_seq = Amino_acid_to_Codon(test_amino_seq)
codon_seq()

[['AUG'], ['CAA', 'CAG'], ['AGC', 'AGU', 'UCA', 'UCC', 'UCG', 'UCU']]

Codon Seq. from RNA basis to DNA basis

In [4]:
codon_seq_in_dna_base = codon_seq.in_dna_base()
print(codon_seq())
print(codon_seq_in_dna_base)

[['AUG'], ['CAA', 'CAG'], ['AGC', 'AGU', 'UCA', 'UCC', 'UCG', 'UCU']]
[['ATG'], ['CAA', 'CAG'], ['AGC', 'AGT', 'TCA', 'TCC', 'TCG', 'TCT']]


---
# Codon_Hamiltonian

parameters

In [5]:
weight_params = {'c_f': 0.1, 'c_GC': 1, 'c_R': 0.1, 'c_L': 1, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 1, 'infty': 10}
H_codon = Codon_Hamiltonian(test_amino_seq, weight_params)

## 1. Incorporating codon usage bias

$$
\mathcal{H}_f = c_f\sum^N_i\log\left[C_i + \varepsilon_f\right]q_i
$$

codon frequency table

In [6]:
import python_codon_tables as pct
# PRINT THE LIST OF NAMES OF ALL AVAILABLE TABLES
print ('Available tables:', pct.available_codon_tables_names)
table = pct.get_codons_table('e_coli_316407')
table['A']

#Table
col1 = test_amino_seq
col2 = [table[x] for x in test_amino_seq]
two_dim_list = [[col1[x], col2[x]] for x in range(len(test_amino_seq))]


my_data = pd.DataFrame(two_dim_list, columns = ['Amino acid sequence', 'Frequences of all possible codons'])
print(my_data)

Available tables: ['b_subtilis_1423', 'd_melanogaster_7227', 'm_musculus_domesticus_10092', 'm_musculus_10090', 'e_coli_316407', 'g_gallus_9031', 'c_elegans_6239', 's_cerevisiae_4932', 'h_sapiens_9606']
  Amino acid sequence                  Frequences of all possible codons
0                   M                                       {'ATG': 1.0}
1                   Q                         {'CAA': 0.35, 'CAG': 0.65}
2                   S  {'AGC': 0.28, 'AGT': 0.15, 'TCA': 0.12, 'TCC':...


$\vec{\zeta}$, where $\zeta_i = \log(C_i+\varepsilon_f)$

In [8]:
H_codon.vec_zeta(epsilon_f=0)

array([ 0.        , -1.04982212, -0.43078292, -1.27296568, -1.89711998,
       -2.12026354, -1.89711998, -1.89711998, -1.89711998])

$\mathcal{H}_f$

In [9]:
H_codon.H_f

array([-9.99500333e-05,  1.04696906e-01,  4.29245637e-02,  1.26940061e-01,
        1.89047544e-01,  2.11196473e-01,  1.89047544e-01,  1.89047544e-01,
        1.89047544e-01])

## 2. Optimize target GC concentration

$$
\rho_{GC} = \frac{1}{N}\sum^N_{i} s_i q_i,
$$
where $s_i$: the number of G's and C's in codon $i$.

$\vec{s}$

In [10]:
H_codon.vec_s()

array([1, 1, 2, 2, 1, 1, 2, 2, 1])

$\vec{s}\otimes\vec{s}$

In [11]:
upper_trianglular_part, diagonal_part = H_codon.matrix_ss()

$$
\mathcal{H}_{GC} = \frac{2c_{GC}}{N^2}\sum^N_i\sum^N_{j<i} s_is_jq_iq_j + \sum^N_i \left(\frac{c_{GC}}{N^2}s^2_i - \frac{2\rho_Tc_{GC}}{N}s_i\right)q_i + c_{GC}\rho^2_T
$$

In [12]:
quadratic_coeff, linear_coeff, const = H_codon.H_GC

In [13]:
np.max(quadratic_coeff)

-0.09876543209876543

## 3. Minimize sequentially repeated nuceotides

$$
\mathcal{H}_R = c_R\sum^N_i\sum^{N}_{j<i} R_{ij}q_iq_j,
$$
$$
R_{ij} = r(C_i, C_J) \kappa_{i,j}
$$

$r(C_i, C_j)$

In [14]:
# r(C_i, C_j)
def repeated_sequential_nucleotides(Ci, Cj):
    input = Ci + Cj
    groups = groupby(input)
    result = [(label, len(list(group))) for label, group in groups]
    list_counts = np.array(result)[:,1]
    outcome = np.max(list_counts.astype('int'))
    return outcome ** 2 - 1

In [15]:
H_codon._repeated_sequential_nucleotides('AUG', 'CAA')

3

$R_{ij}$

In [16]:
H_codon.matrix_R()

array([[0., 3., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 8., 8., 3., 3., 3., 3.],
       [0., 0., 0., 0., 0., 0., 3., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [17]:
np.max(H_codon.matrix_R())

8.0

## 4. Additional constraints

$$
\mathcal{H}_p = -\sum^N_i \epsilon q_i + \sum^N_i\sum^N_{j<i} \tau_{ij}q_iq_j
$$

In [18]:
H_codon.matrix_tau()

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 50., 50., 50., 50., 50.],
       [ 0.,  0.,  0.,  0.,  0., 50., 50., 50., 50.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 50., 50., 50.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 50., 50.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 50.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

## 5. Conservation of the length of polypeptide

$$
\mathcal{H}_L = c_L\left(1-\frac{1}{l}\sum^N_{i=1}q_i\right)^2
= c_L\left[1 + \left(\frac{1}{l^2}-\frac{2}{l}\right)\sum^N_{i=1} q_i + \frac{2}{l^2}\sum^N_{i=1}\sum^{N}_{i<j}q_iq_j \right]
$$

$$
L_{ii} = \left(\frac{1}{l^2}-\frac{2}{l}\right)I \\ 
L_{ij} = \frac{2}{l^2} \text{ for } i<j
$$

In [19]:
H_codon.matrix_L()

[array([-0.55555556, -0.55555556, -0.55555556, -0.55555556, -0.55555556,
        -0.55555556, -0.55555556, -0.55555556, -0.55555556]),
 array([[0.        , 0.22222222, 0.22222222, 0.22222222, 0.22222222,
         0.22222222, 0.22222222, 0.22222222, 0.22222222],
        [0.        , 0.        , 0.22222222, 0.22222222, 0.22222222,
         0.22222222, 0.22222222, 0.22222222, 0.22222222],
        [0.        , 0.        , 0.        , 0.22222222, 0.22222222,
         0.22222222, 0.22222222, 0.22222222, 0.22222222],
        [0.        , 0.        , 0.        , 0.        , 0.22222222,
         0.22222222, 0.22222222, 0.22222222, 0.22222222],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.22222222, 0.22222222, 0.22222222, 0.22222222],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.22222222, 0.22222222, 0.22222222],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

## 6. Codon Hamiltonian

$$
\mathcal{H} = \sum^N_i Q_{ii} q_i + \sum^N_i\sum^N_{j<i}Q_{ij}q_iq_j + c_{GC}\rho^2_T.
$$

$$
Q_{ii} = c_f\zeta_i - \frac{2\rho_Tc_{GC}}{N}s_i + \frac{c_{GC}}{N^2}s^2_i - \varepsilon + c_LL_{i}
$$
$$
Q_{ij} = \frac{2c_{GC}}{N^2}\sigma_{ij} + c_R R_{ij} + \tau_{ij} + c_L L_{ij}
$$

In [29]:
np.max(H_codon.Q_ii), np.min(H_codon.Q_ii)

(-1.4431245143157814, -1.6854704980510382)

In [30]:
np.max(H_codon.Q_ij), np.min(H_codon.Q_ij)

(10.320987654320987, 0.0)

---
# Simulating a Toy Model 

test input

In [2]:
#spike_sars2_seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT'
A0A2U1LIM9 = 'MQSTTSVKLSPFDLMTALLNGKVSFDTSNTSDTNIPLAVFMENRELLMILTTSVAVLIGCVVVLVWRRSSSAAKKAAESPVIVVPKKVTEDEVDDGRKKVTVFFGTQTGTAEGFAKALVEEAKARYEKAVFKVIDLDDYAAEDDEYEEKLKKESLAFFFLATYGDGEPTDNAARFYKWFTEGEEKGEWLEKLQYAVFGLGNRQYEHFNKIAKVVDEKLVEQGAKRLVPVGMGDDDQCIEDDFTAWKELVWPELDQLLRDEDDTSVATPYTAAVAEYRVVFHDKPETYDQDQLTNGHAVHDAQHPCRSNVAVKKELHSPLSDRSCTHLEFDISNTGLSYETGDHVGVYVENLSEVVDEAEKLIGLPPHTYFSVHTDNEDGTPLGGASLPPPFPPCTLRKALASYADVLSSPKKSALLALAAHATDSTEADRLKFLASPAGKDEYAQWIVASHRSLLEVMEAFPSAKPPLGVFFASVAPRLQPRYYSISSSPKFAPNRIHVTCALVYEQTPSGRVHKGVCSTWMKNAVPMTESQDCSWAPIYVRTSNFRLPSDPKVPVIMIGPGTGLAPFRGFLQERLAQKEAGTELGTAILFFGCRNRKVDFIYEDELNNFVETGALSELVTAFSREGATKEYVQHKMTQKASDIWNLLSEGAYLYVCGDAKGMAKDVHRTLHTIVQEQGSLDSSKAELYVKNLQMAGRYLRDVW'
HPDF_amino = 'EGPALRRSYWRHLRRLVLGPPEPPFSHVCQVGDPVLRGVAAPVERAQLGGPELQRLTQRLVQVMRRRRCVGLSAPQLGVPRQVLALELPEALCRECPPRQRALRQMEPFPLRVFVNPSLRVLDSRLVTFPEGCESVAGFLACVPRFQAVQISGLDPNGEQVVWQASGWAARIIQHEMDHLQGCLFIDKMDSRTFTNVYWMKVND'

In [16]:
amino_seq = fragmenting_amino_acid_seq(HPDF_amino, 3, 0)
codon_seq = Amino_acid_to_Codon(amino_seq)
print('Amino acid seq:', amino_seq)
print('All possible codons:', codon_seq())

Amino acid seq: EGP
All possible codons: [['GAA', 'GAG'], ['GGA', 'GGC', 'GGG', 'GGU'], ['CCA', 'CCC', 'CCG', 'CCU']]


Preparing H_codon

In [31]:
weight_params = {'c_f': 0.1, 'c_GC': 1, 'c_R': 0.1, 'c_L': 1, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 1, 'infty': 10}
H_codon = Codon_Hamiltonian(amino_seq, weight_params)

## 1. D-wave

Constructing Q_{ij} as dict type

In [4]:
Q = H_codon.get_Q_dict()

### Simulation

In [5]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

#import matplotlib
#matplotlib.use("agg")
#from matplotlib import pyplot as plt

In [6]:
# ------- Run our QUBO on the QPU -------
# Set up QPU parameters
#이건 우리가 최적화 해야할듯?
chainstrength = 15 #min: 12
numruns = 10000

In [7]:
# Run the QUBO on the solver from your config file
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               #num_spin_reversal_tramsforms=50,
                               label='Codon_sequence')

In [8]:
def get_min_res(response):
    # Pick minimum Energy and its index
    min_E = min(list(response.data(fields=['energy'])))[0]
    min_index = list(response.data(fields=['energy'])).index(min_E)
    # Pick a sample associated with min Energy
    min_sample = list(response.data(fields=['sample']))[min_index][0]
    min_sample = [k for k,v in min_sample.items() if v == 1]
    
    return min_sample, min_E

In [9]:
get_min_res(response)

([0, 2, 3, 11], -4.948192389640039)

In [10]:
print('-' * 60)
print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
print('-' * 60)
for sample, E in response.data(fields=['sample','energy']):
    S0 = [k for k,v in sample.items() if v == 0]
    S1 = [k for k,v in sample.items() if v == 1]
    print('{:>15s}{:^15s}{:^15s}'.format(str(S1),str(E),str(int(-1*E))))

------------------------------------------------------------
          Set 0          Set 1    Energy        Cut Size    
------------------------------------------------------------
  [0, 2, 3, 11]-4.948192389640039       4       
  [0, 2, 3, 11]-4.948192389640039       4       
  [0, 2, 3, 12]-4.896120944050517       4       
  [0, 2, 8, 11]-4.886084906388158       4       
  [0, 2, 4, 11]-4.886084906388158       4       
  [0, 2, 7, 11]-4.886084906388157       4       
   [0, 2, 3, 9]-4.875500239872185       4       
   [0, 2, 3, 9]-4.875500239872185       4       
   [0, 2, 3, 9]-4.875500239872185       4       
  [0, 2, 7, 12]-4.8340134607986345       4       
  [0, 2, 4, 12]-4.8221791412720085       4       
  [0, 2, 8, 12]-4.8221791412720085       4       
   [0, 2, 7, 9]-4.813392756620304       4       
   [0, 2, 8, 9]-4.801558437093677       4       
   [0, 2, 4, 9]-4.801558437093677       4       
  [0, 2, 3, 10]-4.696884995096754       4       
  [0, 2, 4, 10]-4.634777511844

new $\mathcal{H}_L$

[0, 2, 3, 11, 16, 17, 25]-7.537854785825701 <br>
[0, 2, 4, 11, 15, 17, 25]-7.527818748163343  <br>

[0, 2, 8, 12, 15, 17, 25, 28]-8.02910625876468

[0, 2, 3, 11, 15, 17, 25]-6.592669715640199        <br>

[0, 2, 3, 11, 15, 17, 25, 28]-7.150419551458638  <br>
[0, 2, 3, 11, 15, 17, 24, 28]-7.0986156153496704       <br>
[0, 2, 4, 11, 15, 17, 25, 28]-7.085933946922571  <br>
[0, 2, 4, 11, 15, 17, 24, 28]-7.034130010813604 

In [11]:
H_codon.list_all_possible_codons

[['AUG'],
 ['CAA', 'CAG'],
 ['AGC', 'AGU', 'UCA', 'UCC', 'UCG', 'UCU'],
 ['ACA', 'ACC', 'ACG', 'ACU']]

In [12]:
H_codon.run_Dwave()

([0, 2, 3, 11], -4.948192389640039)

In [14]:
H_codon.opt_codon_seq(base='DNA')

['ATG', 'CAG', 'AGC', 'ACG']

## 2. Exact Diagonalization

Constructing Ising model from QUBO & Running 

In [32]:
J, h, const = H_codon.Q_to_Jh()

In [33]:
tests = Quantum_Ising(J=J, h=h, shift=const)
tests.hamiltonian

array([[-25.59094802,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        , -27.41729893,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , -26.22394128, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,  61.22394128,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
         62.41729893,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,  90.59094802]])

In [34]:
GS = tests.ExactDiag()
vec_to_braket(GS)

{'|0110000100>': 1.0}

In [35]:
tests.GE

-5.4781521778580995

---
# Run Simulator

In [2]:
HPDF_amino = 'EGPALRRSYWRHLRRLVLGPPEPPFSHVCQVGDPVLRGVAAPVERAQLGGPELQRLTQRLVQVMRRRRCVGLSAPQLGVPRQVLALELPEALCRECPPRQRALRQMEPFPLRVFVNPSLRVLDSRLVTFPEGCESVAGFLACVPRFQAVQISGLDPNGEQVVWQASGWAARIIQHEMDHLQGCLFIDKMDSRTFTNVYWMKVND'
print('Length of full seq:', len(HPDF_amino))

Length of full seq: 204


## ED

### 1. Run all

In [54]:
weight_params = {'c_f': 1, 'c_GC': 0, 'c_R': 0, 'c_L': 0, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 2, 'infty': 4}

In [ ]:
block_size = 2
verbose = 0

aminoacid_block = []
ED_codons = []
min_E_list = []

print("* Running", end=' ') if verbose == 0 else None
for ith in range(len(HPDF_amino) // block_size + 1):
    amino_fragment = fragmenting_amino_acid_seq(HPDF_amino, block_size, ith)
    codon_fragment = Amino_acid_to_Codon(amino_fragment)
        
    if verbose >= 1:
        print('In amino acide seq, Run Block:',str(ith))
        print('=> Amino acids:', amino_fragment)
    if verbose >= 2:
        print('=> All possible codons:', codon_fragment())
    #====================
    
    #construct H_codon
    H_codon = Codon_Hamiltonian(amino_fragment, weight_params)
    J, h, const = H_codon.Q_to_Jh()
    #run Exact Diagonalization
    H_codon = Quantum_Ising(J=J, h=h, shift=const)
    H_codon.run_ExactDiag()
    select_q1 = H_codon.get_outcome(types='list')
    opt_codon_frag = codon_fragment(select_q1)
    
    #====================
    if verbose ==0:
        print(".", end=' ')
    elif verbose >= 2:
        print('=> Ground states:', min_sample)
    elif verbose >= 1:
        print('=> Optimal codons:', opt_codon_frag)


    aminoacid_block.append(amino_fragment)
    ED_codons.append(opt_codon_frag)
    min_E_list.append(H_codon.ground_energy)


In [57]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

opt_codon_list = ["".join(ED_codons[x]) for x in range(len(ED_codons))]
rslt_df = pd.DataFrame(list(zip(aminoacid_block, opt_codon_list, min_E_list)),
                        columns=['amino acid', 'codon', "Energy"])
rslt_df

,amino acid,codon,Energy
0,EG,GAAGGC,-2.741222
1,PA,CCGGCG,-2.348129
2,LR,CUGCGC,-2.395057
3,RS,CGCAGC,-1.816806
4,YW,UAUUGG,-3.440633
5,RH,CGCCAU,-2.525840
6,LR,CUGCGC,-2.395057
7,RL,CGCCUG,-2.395057
8,VL,GUGCUG,-2.317298
9,GP,GGCCCG,-2.477845


### 2. Run_ED for just one frag

#### TEST1:  Codon Usage Frequency

$c_f = 1$ is fixed. <br>
$\infty = 2 * \epsilon$ <br>
block size = $\epsilon$


In [48]:
weight_params = {'c_f': 1, 'c_GC': 0, 'c_R': 0, 'c_L': 0, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 2, 'infty': 4}
# epsilon

In [53]:
block_size = 2
block_position = 0

amino_fragment = fragmenting_amino_acid_seq(HPDF_amino, block_size, block_position)
print('=> Amino acids:', amino_fragment)
codon_fragment = Amino_acid_to_Codon(amino_fragment)
print('=> All possible codons:', codon_fragment())


H_codon = Codon_Hamiltonian(amino_fragment, weight_params)
J, h, const = H_codon.Q_to_Jh()
H_codon_matrix = Quantum_Ising(J=J, h=h, shift=0) #shift=const
H_codon_matrix.run_ExactDiag()

# Pick sites where qubit=1
select_q1 = H_codon_matrix.get_outcome(types='list')
codon_fragment(select_q1)

=> Amino acids: EG
=> All possible codons: [['GAA', 'GAG'], ['GGA', 'GGC', 'GGG', 'GGU']]


['GAA', 'GGC']

In [50]:
H_codon.Q_to_Jh()

(array([[0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([-2.82774093, -3.05598237, -2.31649663, -2.91317546, -2.77794857,
        -2.65281823, -2.50943866, -2.91317546]),
 9.966776297157107)

In [51]:
print(H_codon_matrix.ground_energy)
#H_codon_matrix.hamiltonian

-12.314905718767651


In [52]:
H_codon_matrix.run_ExactDiag()
GS = H_codon_matrix.ground_state
if len(GS.shape) == 1:
    print(vec_to_braket(GS))
else:
    for i in range(len(GS)):
        print(vec_to_braket(GS[i]))

{'|00100010>': 1.0}


#### TEST2: GC Contents (Not Yet!)

In [53]:
weight_params = {'c_f': 1, 'c_GC': 0, 'c_R': 0, 'c_L': 0, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 1, 'infty': 2}

In [60]:
block_size = 1
block_position = 0

amino_fragment = fragmenting_amino_acid_seq(HPDF_amino, block_size, block_position)
print('=> Amino acids:', amino_fragment)
codon_fragment = Amino_acid_to_Codon(amino_fragment)
print('=> All possible codons:', codon_fragment())


H_codon = Codon_Hamiltonian(amino_fragment, weight_params)
J, h, const = H_codon.Q_to_Jh()
H_codon_matrix = Quantum_Ising(J=J, h=h, shift=0) #shift=const
H_codon_matrix.run_ExactDiag()

# Pick sites where qubit=1
select_q1 = H_codon_matrix.get_outcome(types='list')
codon_fragment(select_q1)

=> Amino acids: E
=> All possible codons: [['GAA', 'GAG']]


[]

In [66]:
H_codon.vec_zeta(epsilon_f=.001)

array([-0.36961546, -1.16796237])

In [62]:
H_codon.Q_ii

array([0.36961546, 1.16796237])

In [63]:
H_codon.Q_to_Jh()

(array([[0., 0.],
        [0., 0.]]),
 array([-0.18480773, -0.58398118]),
 0.7687889110086851)

In [64]:
print(H_codon_matrix.ground_energy)
H_codon_matrix.hamiltonian

-0.7687889110086851


array([[-0.76878891,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.39917346,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.39917346,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.76878891]])

In [65]:
GS = H_codon_matrix.ground_state
vec_to_braket(GS)

{'|00>': 1.0}

## D-wave

In [11]:
HPDF_amino = 'EGPALRRSYWRHLRRLVLGPPEPPFSHVCQVGDPVLRGVAAPVERAQLGGPELQRLTQRLVQVMRRRRCVGLSAPQLGVPRQVLALELPEALCRECPPRQRALRQMEPFPLRVFVNPSLRVLDSRLVTFPEGCESVAGFLACVPRFQAVQISGLDPNGEQVVWQASGWAARIIQHEMDHLQGCLFIDKMDSRTFTNVYWMKVND'
print('Length of full seq:', len(HPDF_amino))





Length of full seq: 204


### 1. Run_Dwave for just one frag

#### TEST1: Codon Usage Frequency

In [58]:
weight_params = {'c_f': 1, 'c_GC': 0, 'c_R': 0, 'c_L': 0, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 2, 'infty': 4}
# epsilon

In [62]:
block_size = 3
block_position = 1

amino_fragment = fragmenting_amino_acid_seq(HPDF_amino, block_size, block_position)
print('=> Amino acids:', amino_fragment)
codon_fragment = Amino_acid_to_Codon(amino_fragment)
print('=> All possible codons:', codon_fragment())


H_codon = Codon_Hamiltonian(amino_fragment, weight_params)
_, min_E = H_codon.run_Dwave(chain_strength=17)
H_codon.outcome_codon_seq()

=> Amino acids: ALR
=> All possible codons: [['GCA', 'GCC', 'GCG', 'GCU'], ['CUA', 'CUC', 'CUG', 'CUU', 'UUA', 'UUG'], ['AGA', 'AGG', 'CGA', 'CGC', 'CGG', 'CGU']]


['GCG', 'CUG', 'CGC']

### 2. Fully run

In [63]:
weight_params = {'c_f': 0.1, 'c_GC': 1, 'c_R': 0.1, 'c_L': 0, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 1, 'infty': 10}
weight_params = {'c_f': 1, 'c_GC': 0, 'c_R': 0, 'c_L': 0, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 2, 'infty': 4}

In [ ]:
block_size = 3
verbose = 0

aminoacid_block = []
dwave_opt_codons = []
min_E_list = []

print("* Running", end=' ') if verbose == 0 else None
for ith in range(len(HPDF_amino) // block_size + 1):
    amino_fragment = fragmenting_amino_acid_seq(HPDF_amino, block_size, ith)
    codon_fragment = Amino_acid_to_Codon(amino_fragment)

    if verbose >= 1:
        print('In amino acide seq, Run Block:',str(ith))
        print('=> Amino acids:', amino_fragment)
    if verbose >= 2:
        print('=> All possible codons:', codon_fragment())
    #====================

    #construct H_codon
    H_codon = Codon_Hamiltonian(amino_fragment, weight_params)
    #run Dwave Sampler
    min_sample, min_E = H_codon.run_Dwave() #chain_strength=15
    opt_codon_frag = H_codon.outcome_codon_seq()

    #====================
    if verbose ==0:
        print(".", end=' ')
    elif verbose >= 2:
        print('=> Ground states:', min_sample)
    elif verbose >= 1:
        print('=> Optimal codons:', opt_codon_frag)



    aminoacid_block.append(amino_fragment)
    dwave_opt_codons.append(opt_codon_frag)
    min_E_list.append(min_E)


#### Outcomes

In [15]:
dwave_opt_codon_list = ["".join(dwave_opt_codons[x]) for x in range(len(dwave_opt_codons))]
rslt_df = pd.DataFrame(list(zip(aminoacid_block, dwave_opt_codon_list, min_E_list)),
                        columns=['amino acid', 'codon', "Energy"])
rslt_df

,amino acid,codon,Energy
0,EGP,,0.000000
1,ALR,,0.000000
2,RSY,,0.000000
3,WRH,UGG,-0.001000
4,LRR,CUG,0.691149
...,...,...,...
63,DSR,,0.000000
64,TFT,,0.000000
65,NVY,,0.000000
66,WMK,UGGAUG,-0.001999


#### Save

In [ ]:
weight_params = {'c_f': 1, 'c_GC': 0, 'c_R': 0, 'c_L': 0, 'epsilon_f': 0.001, 'rho_T': 0.5, 'epsilon': 0, 'infty': 0}

In [8]:
DNA='HPDF'
filepath=f'./result_data/'
os.makedirs(filepath, exist_ok=True)

if weight_params['c_L'] == 0.0: 
    rslt_df.to_csv(os.path.join(filepath, f'{DNA}_bs{block_size}_woL.csv'))
else:
    rslt_df.to_csv(os.path.join(filepath, f'{DNA}_bs{block_size}_wL.csv'))


#### Metrics

In [10]:
dwave_codon_RNA = "".join(rslt_df['codon'])
dwave_codon_DNA = dwave_codon_RNA.replace("U", "T")

In [81]:
#CAI
CAI_hu = CAIs("human")
CAI_ec = CAIs("ecoli")

CAI_hu(dwave_codon_DNA)
CAI_ec(dwave_codon_DNA)

print("-"*30)
print("dwave_codon (block-size:5)")
print(f"CAI of dwave_codon for human : {CAI_hu(dwave_codon_DNA)}")
print(f"CAI of dwave_codon for ecoli : {CAI_ec(dwave_codon_DNA)}")

#GC
from Bio.SeqUtils import GC
GC_dwave = GC(dwave_codon_DNA)
print(f"GC of dwave_codon : {GC_dwave}")

------------------------------
dwave_codon (block-size:5)
CAI of dwave_codon for human : 0.6806920461771745
CAI of dwave_codon for ecoli : 0.7994921955064357
GC of dwave_codon : 65.50580431177445


---

In [43]:
df_opt_codon = pd.read_csv('./jhlee/rslt/HPDF/HPDF_cS5_nT1_woL.csv', index_col=0)
dwave_codon_RNA = "".join(df_opt_codon['codon'])
dwave_codon_DNA = dwave_codon_RNA.replace("U", "T")

pct_ec = getFlatPct("ecoli")
pct_hu = getFlatPct("human")

rscu_hu = get_RSCU_from_ctable(pct_hu)
rscu_ec = get_RSCU_from_ctable(pct_ec)

CAI_hu_opt_dwave = CAI(dwave_codon_DNA, RSCUs=rscu_hu)
CAI_ec_opt_dwave = CAI(dwave_codon_DNA, RSCUs=rscu_ec)

print("-"*30)
print("dwave_codon (block-size:5)")
print(f"CAI of dwave_codon for human: {CAI_hu_opt_dwave}")
print(f"CAI of dwave_codon for ecoli: {CAI_ec_opt_dwave}")


------------------------------
dwave_codon (block-size:5)
CAI of dwave_codon for human: 0.6577886323903238
CAI of dwave_codon for ecoli: 0.8123250512375765


---
# Results (CAI & GC)

1. reference codon DNA <br>
BiLSTM codon DNA

In [26]:
from Bio.SeqUtils import GC
ref_codon_DNA = 'GAGGGCCCGGCGCTGCGGCGCTCCTATTGGCGCCACCTGAGGCGTCTGGTGCTGGGTCCTCCCGAACCGCCGTTCTCGCACGTGTGCCAAGTCGGGGACCCGGTGCTGCGCGGCGTGGCGGCCCCGGTGGAGCGGGCGCAGCTAGGCGGGCCCGAGCTGCAGCGGCTGACGCAACGGCTGGTCCAGGTGATGCGGCGGCGGCGCTGCGTGGGCCTAAGCGCGCCGCAGCTGGGGGTGCCGCGGCAGGTGCTGGCGCTGGAGCTCCCCGAGGCGCTGTGTCGGGAGTGCCCGCCCCGCCAGCGCGCGCTCCGCCAAATGGAGCCCTTCCCCCTGCGCGTGTTCGTGAACCCCAGCCTGCGAGTGCTTGACAGCCGCCTGGTCACCTTTCCCGAGGGCTGCGAGAGCGTCGCCGGCTTCCTGGCCTGCGTGCCCCGCTTCCAGGCGGTGCAGATCTCAGGGCTGGACCCCAATGGAGAACAGGTGGTGTGGCAGGCGAGCGGGTGGGCAGCCCGCATCATCCAGCACGAGATGGACCACCTGCAGGGCTGCCTGTTTATTGACAAAATGGACAGCAGGACGTTCACAAACGTCTATTGGATGAAGGTGAATGACTAA'
BiLSTM_codon_DNA = 'GAAGGTCCGGCACTGCGTCGTAGCTATTGGCGTCATCTGCGTCGTCTGGTGCTGGGTCCGCCGGAACCGCCGTTTAGCCACGTGTGCCAGGTCGGCGATCCGGTGCTGCGTGGCGTGGCGGCACCGGTGGAACGTGCGCAGCTGGGCGGTCCGGAACTGCAGCGTCTGACCCAGCGTCTGGTGCAGGTGATGCGTCGTCGTCGTTGCGTCGGTCTGAGCGCACCGCAGCTGGGCGTGCCGCGTCAGGTGCTGGCGCTGGAACTGCCGGAAGCGCTGTGCCGTGAATGCCCGCCGCGTCAGCGTGCGCTGCGTCAGATGGAACCGTTTCCGCTGCGCGTGTTTGTTAACCCGAGCCTGCGCGTGCTGGATAGCCGTCTGGTGACCTTTCCGGAAGGCTGCGAAAGCGTGGCGGGTTTTCTGGCGTGCGTGCCGCGTTTTCAGGCGGTGCAGATTTCCGGTCTGGATCCGAACGGCGAACAGGTGGTGTGGCAGGCGAGCGGCTGGGCGGCGCGTATTATTCAGCATGAAATGGATCATCTGCAGGGCTGCCTGTTTATTGATAAAATGGATAGCCGTACCTTTACCAACGTCTATTGGATGAAAGTGAACGATTAA'

In [27]:
CAI_hu = CAIs("human")
CAI_ec = CAIs("ecoli")

print("-"*30)
print("reference_codon")
print(f"CAI of ref_codon for human: {CAI_hu(ref_codon_DNA)}")
print(f"CAI of ref_codon for ecoli: {CAI_ec(ref_codon_DNA)}")
print(f"GC of ref_codon: {GC(ref_codon_DNA)}")
print("-"*30)
print("BiLSTM_codon")
print(f"CAI of BiLSTM_opt_codon for human: {CAI_hu(BiLSTM_codon_DNA)}")
print(f"CAI of BiLSTM_opt_codon for ecoli: {CAI_ec(BiLSTM_codon_DNA)}")
print(f"GC of BiLSTM_opt_codon: {GC(BiLSTM_codon_DNA)}")


------------------------------
reference_codon
CAI of ref_codon for human: 0.7876879260559179
CAI of ref_codon for ecoli: 0.6626674257312732
GC of ref_codon: 69.4308943089431
------------------------------
BiLSTM_codon
CAI of BiLSTM_opt_codon for human: 0.682288617477938
CAI of BiLSTM_opt_codon for ecoli: 0.9662881947629263
GC of BiLSTM_opt_codon: 62.4390243902439


2. Dwave codon DNA with default weight params

In [93]:
list_dwave_codon = []
for x in [3, 5, 40]:
    df_opt_codon = pd.read_csv('./result_data/HPDF_bs'+str(x)+'_woL.csv', index_col=0)
    dwave_codon_DNA = ''.join(df_opt_codon['codon']).replace("U", "T")
    



'GAAGGTCCGGCGCTGCGTCGCAGCTATTGGCGTCATCTGCGTCGCCTCGTGCTCGGACCGCCGGAACCGCCGTTCTCGCATGTGTGTCAGGTTGGCGATCCGGTGCTGCGTGGCGTCGCGGCGCCGGTCGAGCGTGCGCAGCTCGGCGGTCCAGAGCTGCAGCGCCTGACACAGCGTCTGGTGCAGGTGATGCGTCGTCGTCGCTGCGTGGGCCTGAGCGCACCGCAGCTCGGCGTTCCGCGTCAGGTGCTCGCACTCGAGCTGCCGGAAGCGCTGTGTCGCGAGTGTCCGCCGCGTCAGCGCGCTCTGCGTCAGATGGAACCGCCGCTGCGCGTGTTCGTGAATCCGTCTCTGCGCGTGCTCGATAGTCGCCTCGTGACGCCGGAAGGCTGCGAGAGCGTCGCAGGCTTCCTGGCGTGCGTGCCGCGTTTCCAGGCAGTGCAGATCAGCGGCCTCGATCCGAATGGTGAGCAGGTCGTGTGGCAGGCGAGCGGCGCAGCGCGCATCATACAGCATGAGATGGATCATCTGCAAGGCTGTCTGTTCATCGATAAGATGGATTCGCGCACGTTCACCAACGTGTATTGGATGAAGGTGAACGAT'

In [40]:
df_opt_codon = pd.read_csv('./result_data/HPDF_bs5_woL.csv', index_col=0)
dwave_codon_RNA_3 = ''.join(df_opt_codon['codon'])
dwave_codon_DNA_3 = dwave_codon_RNA_3.replace("U", "T")

CAI_hu_opt_dwave_3 = CAI(dwave_codon_DNA_3, RSCUs=rscu_hu)
CAI_ec_opt_dwave_3 = CAI(dwave_codon_DNA_3, RSCUs=rscu_ec)


dwave_codon_RNA_5 = ''.join(sum(dwave_opt_codons_5, []))
dwave_codon_DNA_5 = dwave_codon_RNA_5.replace("U", "T")

CAI_hu_opt_dwave_5 = CAI(dwave_codon_DNA_5, RSCUs=rscu_hu)
CAI_ec_opt_dwave_5 = CAI(dwave_codon_DNA_5, RSCUs=rscu_ec)





print("-"*30)
print("dwave_codon (block-size:3)")
print(f"CAI of dwave_codon for human: {CAI_hu_opt_dwave_3}")
print(f"CAI of dwave_codon for ecoli: {CAI_ec_opt_dwave_3}")

print("-"*30)
print("dwave_codon (block-size:5)")
print(f"CAI of dwave_codon for human: {CAI_hu_opt_dwave_5}")
print(f"CAI of dwave_codon for ecoli: {CAI_ec_opt_dwave_5}")



## can compare CAI result from genscript website
# genscript web link: https://www.genscript.com/tools/rare-codon-analysis
# biologics web link: https://www.biologicscorp.com/tools/RareCodonAnalyzer/#.YdZJfd_P1D8

NameError: name 'dwave_opt_codons_3' is not defined